In [8]:
import requests
import re
from common import *
import pandas as pd
from utils import *
from Parallelizer import make_parallel
from yaml.loader import SafeLoader
import yaml
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
def get_info(username):
    url = f"https://tokapi-mobile-version.p.rapidapi.com/v1/user/{username}"

    headers = {
        "X-RapidAPI-Key": os.getenv("RAPIDAPI_KEY"),
        "X-RapidAPI-Host": "tokapi-mobile-version.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers)

    return response.json()

def get_posts(user_id):
    url = f"https://tokapi-mobile-version.p.rapidapi.com/v1/post/user/{user_id}/posts"

    querystring = {"offset":"0","count":"10","region":"VN","with_pinned_posts":"1"}

    headers = {
        "X-RapidAPI-Key": os.getenv("RAPIDAPI_KEY"),
        "X-RapidAPI-Host": "tokapi-mobile-version.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    return response.json()

def url2username(url):
    username = url.split("/")[-1]
    if "MS4wLjA" in username:
        return username[1:]
    else:
        return username

MAPPING_POST = {
    "share_url": ["share_url"],
    "create_time": ["create_time"],
}
@make_parallel
def post_transform(post):
    return Common.mapping_data(post, MAPPING_POST)


MAPPING = {
    "username": ["user","unique_id"],
    "follower": ["user","follower_count"],
    "bio_url": ["user","bio_url"],
    "bio": ["user","signature"],
    "sec_uid": ["user","sec_uid"],
    "has_shop": ["user","tab_settings","shop_tab","show_shop_tab"],
    "uid": ["user","uid"],
}

In [13]:
os.environ["RAPIDAPI_KEY"]="d1583bfe26msh53d8ca2e32d95c3p182a27jsn990cb23d366c"

In [4]:
links = input("Enter links: ")
links = links.split(" ")
print(f"Number of links: {len(links)}")
print(f"sample link: {links[:3]}")

Number of links: 3
sample link: ['https://www.tiktok.com/@MS4wLjABAAAAx4dDiQg8yUeraJjUupXkTwSG1IVnD7L4zGM_bb5syBfOYI-FjQSCiNPsue9vE7eS', 'https://www.tiktok.com/@MS4wLjABAAAAMr-ECcuhYi8o84ZjvqMQjrqQxwlITZxq9qBH1xRnKx-d6ulgZsMbgF1v1KJzkkVs', 'https://www.tiktok.com/@MS4wLjABAAAADuLvyxgb54HqyCSek1-P_vD76mpS6YXRylwWa_KLa3mLiTX1p29dDtL1qyNm62tq']


In [15]:
results = []
for link in links:
    transformed = {"url": link}
    username = url2username(link)
    try:
        info = get_info(username)
        transformed_ = Common.mapping_data(info, MAPPING)
        transformed.update(transformed_)
        post_list = get_posts(transformed["uid"])
        try:
            post_list = post_list["aweme_list"]
            post_list_transformed = post_transform(post_list)
            latest_post = max(post_list_transformed, key=lambda x: x["create_time"])
            if latest_post:
                minimal_url = latest_post["share_url"].split("?")[0]
                transformed["latest_post"] = minimal_url
            results.append(transformed)
        except Exception as e:
            print(f"Error getting posts for {username}: {e}")
            results.append(transformed | {"error": str(e)})
    except Exception as e:
        print(f"Error getting info for {username}: {e}")
        results.append({"username": username, "error": str(e)})
    df = pd.DataFrame(results)
    df["follower"] = df["follower"].apply(lambda x: big_number_to_string_number(x))
    # df["other_url"] = df["bio"].apply(lambda x: extract_url(x) if x else None)
    # df["url"] = df["username"].apply(lambda x: f"https://www.tiktok.com/@{x}")
    df["bio_url"] = df["bio_url"].apply(lambda x: x or "")
    df["bio"] = df["bio"] + df["bio_url"]
    df["standard_url"] = df["username"].apply(lambda x: f"https://www.tiktok.com/@{x}")
    cols = ["url","sec_uid","latest_post","follower","has_shop","bio", "standard_url"]
    try:
        df = df[cols]
    except Exception as e:
        print(f"Error selecting columns: {e}")
        
df.to_csv("tiktok.csv", index=False)

'bio_url'
'shop_tab'
'bio_url'
